# Token Usage
Compute the (input- and output) token usage of the different models and datasets over all conducted experiment runs. We also evaluate the fraction of tokens consumed by the meta-llm-

In [1]:
import pandas as pd
from glob import glob

In [8]:
datasets = ["agnews", "sst-5", "gsm8k", "copa", "subj"]
models = ["llama", "qwen", "mistral"]
seeds = [42, 43, 44]
dfs = []
for dataset in datasets:
    for model in models:
        for seed in seeds:
            path = glob(
                f"../results/main_results/{dataset}/{model}/CAPO/seed{seed}/*/*/step_results_eval.csv"
            )[0]
            df = pd.read_csv(path)
            dfs.append(
                df.assign(
                    dataset=dataset,
                    model=model,
                    seed=seed,
                )
            )

df = pd.concat(dfs, ignore_index=True)

In [9]:
# groupby step, dataset, model, seed, calculate cum sum of input_tokens_meta_llm and input_tokens_downstream_llm

df = df.groupby(["dataset", "model", "seed", "step"]).agg(
    input_tokens_meta_llm=("input_tokens_meta_llm", "first"),
    input_tokens_downstream_llm=("input_tokens_downstream_llm", "first"),
    output_tokens_meta_llm=("output_tokens_meta_llm", "first"),
    output_tokens_downstream_llm=("output_tokens_downstream_llm", "first"),
)

df = df.reset_index()
df["input_tokens_meta_llm_cumsum"] = (
    df.groupby(["dataset", "model", "seed"])["input_tokens_meta_llm"].cumsum().astype(int)
)

df["input_tokens_downstream_llm_cumsum"] = (
    df.groupby(["dataset", "model", "seed"])["input_tokens_downstream_llm"].cumsum().astype(int)
)

df["output_tokens_meta_llm_cumsum"] = (
    df.groupby(["dataset", "model", "seed"])["output_tokens_meta_llm"].cumsum().astype(int)
)

df["output_tokens_downstream_llm_cumsum"] = (
    df.groupby(["dataset", "model", "seed"])["output_tokens_downstream_llm"].cumsum().astype(int)
)

In [10]:
df = df.groupby(["dataset", "model", "seed"]).agg(
    input_tokens_meta_llm=("input_tokens_meta_llm_cumsum", "max"),
    input_tokens_downstream_llm=("input_tokens_downstream_llm_cumsum", "max"),
    output_tokens_meta_llm=("output_tokens_meta_llm_cumsum", "max"),
    output_tokens_downstream_llm=("output_tokens_downstream_llm_cumsum", "max"),
)

# build mean
df = df.groupby(["dataset", "model"]).agg(
    input_tokens_downstream_llm=("input_tokens_downstream_llm", "mean"),
    input_tokens_downstream_llm_std=("input_tokens_downstream_llm", "std"),
    input_tokens_meta_llm=("input_tokens_meta_llm", "mean"),
    input_tokens_meta_llm_std=("input_tokens_meta_llm", "std"),
    output_tokens_downstream_llm=("output_tokens_downstream_llm", "mean"),
    output_tokens_downstream_llm_std=("output_tokens_downstream_llm", "std"),
    output_tokens_meta_llm=("output_tokens_meta_llm", "mean"),
    output_tokens_meta_llm_std=("output_tokens_meta_llm", "std"),
)

In [11]:
df["frac_downstream_llm"] = df["input_tokens_downstream_llm"] / (
    df["input_tokens_meta_llm"] + df["input_tokens_downstream_llm"]
)

In [12]:
df

input_tokens_downstream_llm  input_tokens_downstream_llm_std  \
dataset model                                                                   
agnews  llama                   5.059157e+06                     5.934108e+04   
        mistral                 5.099964e+06                     6.262290e+04   
        qwen                    5.040108e+06                     1.376374e+04   
copa    llama                   5.040588e+06                     4.230954e+04   
        mistral                 3.513026e+06                     1.578963e+06   
        qwen                    4.754941e+06                     5.195298e+05   
gsm8k   llama                   5.220248e+06                     2.001068e+05   
        mistral                 5.176992e+06                     1.245152e+05   
        qwen                    5.096732e+06                     4.066272e+04   
sst-5   llama                   5.098120e+06                     6.092621e+04   
        mistral                 5.142890e+06                     1.059569e+04   
        qwen                    5.032211e+06                     3.553257e+04   
subj    llama                   5.070569e+06                     5.838867e+04   
        mistral                 5.109623e+06                     7.747341e+04   
        qwen                    5.082538e+06                     4.751418e+04   

                 input_tokens_meta_llm  input_tokens_meta_llm_std  \
dataset model                                                       
agnews  llama             1.109220e+05               4.083309e+04   
        mistral           6.091900e+04               1.582256e+04   
        qwen              5.279400e+04               2.947583e+04   
copa    llama             8.041433e+04               1.657727e+04   
        mistral           1.271295e+06               1.026011e+06   
        qwen              5.598093e+05               8.259941e+05   
gsm8k   llama             2.104000e+04               4.974822e+03   
        mistral           1.501000e+04               4.050344e+03   
        qwen              2.210967e+04               7.035918e+03   
sst-5   llama             5.742867e+04               2.651516e+04   
        mistral           4.857200e+04               7.914681e+03   
        qwen              4.780267e+04               1.212362e+04   
subj    llama             7.316367e+04               1.879717e+04   
        mistral           5.139533e+04               1.470585e+04   
        qwen              6.309533e+04               2.429697e+04   

                 output_tokens_downstream_llm  \
dataset model                                   
agnews  llama                    1.280261e+06   
        mistral                  4.466473e+05   
        qwen                     2.654410e+05   
copa    llama                    3.497225e+06   
        mistral                  3.165999e+06   
        qwen                     2.682985e+06   
gsm8k   llama                    2.008999e+06   
        mistral                  1.881618e+06   
        qwen                     2.586808e+06   
sst-5   llama                    1.316234e+06   
        mistral                  1.173709e+06   
        qwen                     1.314268e+06   
subj    llama                    1.007411e+06   
        mistral                  9.583553e+05   
        qwen                     2.949987e+05   

                 output_tokens_downstream_llm_std  output_tokens_meta_llm  \
dataset model                                                               
agnews  llama                        6.036737e+05            47068.333333   
        mistral                      8.040423e+04            25632.333333   
        qwen                         4.177487e+04            19586.000000   
copa    llama                        7.134906e+05            38328.333333   
        mistral                      1.222857e+06           507924.000000   
        qwen                         1.518114e+06           183659.333333   
gsm8k   llama  

In [42]:
print(df["frac_downstream_llm"].mul(100).round(1).to_latex())

\begin{tabular}{llr}
\toprule
 &  & frac_downstream_llm \\
dataset & model &  \\
\midrule
\multirow[t]{3}{*}{agnews} & llama & 97.900000 \\
 & mistral & 98.800000 \\
 & qwen & 99.000000 \\
\cline{1-3}
\multirow[t]{3}{*}{copa} & llama & 98.400000 \\
 & mistral & 73.400000 \\
 & qwen & 89.500000 \\
\cline{1-3}
\multirow[t]{3}{*}{gsm8k} & llama & 99.600000 \\
 & mistral & 99.700000 \\
 & qwen & 99.600000 \\
\cline{1-3}
\multirow[t]{3}{*}{sst-5} & llama & 98.900000 \\
 & mistral & 99.100000 \\
 & qwen & 99.100000 \\
\cline{1-3}
\multirow[t]{3}{*}{subj} & llama & 98.600000 \\
 & mistral & 99.000000 \\
 & qwen & 98.800000 \\
\cline{1-3}
\bottomrule
\end{tabular}



In [43]:
df["frac_downstream_llm"].mean()

np.float64(0.966136400759536)